In [1]:
# We can use pre trained glove vector from above link

In [2]:
# We will download this pre trained glove
#Twitter (2B tweets, 27B tokens, 1.2M vocab, uncased, 25d, 50d, 100d, & 200d vectors, 1.42 GB download): glove.twitter.27B.zip

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Embedding, Activation, Dropout 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam

import numpy as np
from numpy import array
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
df= pd.read_csv('twitter4000.csv')
df.head()

,twitts,sentiment
0,is bored and wants to watch a movie any sugge...,0
1,back in miami. waiting to unboard ship,0
2,"@misskpey awwww dnt dis brng bak memoriessss, ...",0
3,ughhh i am so tired blahhhhhhhhh,0
4,@mandagoforth me bad! It's funny though. Zacha...,0


## Processing and Cleaning

In [6]:
contractions= {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "i would",
  "i'd've": "i would have",
  "i'll": "i will",
  "i'll've": "i will have",
  "i'm": "i am",
  "i've": "i have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

In [9]:
%%time
import re

text= ' '.join(df['twitts'])
text= text.split()
freq_comm= pd.Series(text).value_counts()
rare= freq_comm[freq_comm.values==1]

def get_clean_text(x):
    if type(x) is str:
        x=x.lower()
        for key in contractions:
            value= contractions[key]
            x=x.replace(key, value)
            #Removing email, http etc, RT etc from below code
        x= re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)','',x)
        x=re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',x)
        x= re.sub('RT', "",x)
        x= re.sub('[^A-Z a-z]+','',x)
        x=' '.join([t for t in x.split() if t not in rare])
        return x
    else:
        return x

df['twitts']= df['twitts'].apply(lambda x: get_clean_text(x))

Wall time: 352 ms


In [10]:
df['twitts']

0       is bored and wants to watch a movie any sugges...
1                                back in miami waiting to
2       misskpey awwww dnt bak memoriessss i i am sad lol
3                                     ughhh i am so tired
4       mandagoforth me bad it is funny though zachary...
                              ...                        
3995                                               i just
3996               templating works it all has to be done
3997                      mommy just brought me starbucks
3998       omarepps watching you on a house rerunlovin it
3999    thanks for trying to make me smile i will make...
Name: twitts, Length: 4000, dtype: object

In [11]:
df['sentiment'].value_counts()

0    2000
1    2000
Name: sentiment, dtype: int64

In [12]:
text= df['twitts'].tolist()
text[:3]

['is bored and wants to watch a movie any suggestions',
 'back in miami waiting to',
 'misskpey awwww dnt bak memoriessss i i am sad lol']

In [13]:
y= df['sentiment']

In [14]:
token= Tokenizer()
token.fit_on_texts(text)

In [16]:
vocab_size= len(token.word_index)+1
vocab_size

6790

In [17]:
encoded_text = token.texts_to_sequences(text)

In [18]:
print(encoded_text[:3])

[[5, 279, 9, 315, 2, 181, 4, 216, 201, 2297], [50, 11, 1297, 182, 2], [2298, 1085, 655, 1298, 2299, 1, 1, 13, 115, 48]]


In [19]:
max_length= 120
X= pad_sequences(encoded_text, maxlen=max_length, padding='post')

In [20]:
print(X)

[[   5  279    9 ...    0    0    0]
 [  50   11 1297 ...    0    0    0]
 [2298 1085  655 ...    0    0    0]
 ...
 [ 934   22  923 ...    0    0    0]
 [6788  125    7 ...    0    0    0]
 [  89   12  208 ...    0    0    0]]


In [21]:
X.shape

(4000, 120)

# Glove Vectors

In [22]:
glove_vectors= dict()

In [23]:
%%time
file= open('glove.twitter.27B.200d.txt', encoding='utf-8')

for line in file:
    values= line.split()
    word= values[0]
    vectors= np.asarray(values[1:])
    glove_vectors[word]= vectors
file.close()

Wall time: 2min 32s


In [24]:
# Number of unique words
glove_vectors.get('you').shape

(200,)

In [25]:
word_vector_matrix= np.zeros((vocab_size,200))

for word, index in token.word_index.items():
    vector= glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index]= vector
    else:
        print(word)

tommcfly
dougiemcfly
donniewahlberg
kirstiealley
quotthe
peterfacinelli
davidarchie
quoti
youngq
jordanknight
atampt
quotyou
gtlt
songzyuuup
dannygokey
jackalltimelow
ashleytisdale
jonathanrknight
ughhh
twitterberry
misss
shaaqt
cepic
soooooooooooo
spymaster
notbut
sweetkisses
dcdebbie
earthlifeshop
youquot
ltsobgt
wayyy
workno
heidimontag
mampg
seeee
sohotel
xbllygbsn
damohopo
katlb
anddd
rckergirl
mequot
aplusk
wwwtweeteraddercom
paulgdog
hotwords
deltagoodrem
miamiadc
trvsbrkr
thesupergirl
gailporter
jeffreecuntstar
youngcash
misskpey
memoriessss
mandagoforth
kevinmarquis
peterfacinellis
raisenot
stacig
killahzzzz
sfannah
brianquest
garethemery
kimberleymtkg
daniesass
hungrydomaine
quotmermaid
dinnerquot
shvizut
wellat
shirshor
hummm
jchamanes
scaredddd
andrewhuntre
sonjacassella
darenotspeak
mammaj
jkmrprez
yearquot
haemoglobin
stedic
paulmccourt
greenisland
twilightofdoom
krazyfreak
whyhe
xxbrry
brianrubin
lomara
kankzxd
anthothemantho
antonwheel
hanaames
orlax
bastantep
catshatem

lameeee
binnow
pinkynkotb
genevieveeex
tequilasam
sciphys
pudgesemipudge
gljufura
andakilsa
maryrosem
xtara
nickeeeh
talentdmrripley
llauraxxx
graheme
strikeitfierce
kinoflyhigh
danyingling
shaycarl
waitthat
grossumm
clareies
ginilovesjonas
yesterdaybut
godiegogo
everfact
holidaysugh
doeven
grawrwhy
digitalbritain
bettermzansi
quotdress
codequot
quottrampsquot
epyonic
juliancheal
sswan
ghostfreehood
greenxpeen
dynamiks
omgahitskasey
longsine
internetcomputer
failuredid
winecountrydog
officialtila
lbbchat
harizadri
greekshow
wsendoutscuds
soooolet
shelbyrayne
meticulouzity
lyssajo
catchthesunx
skdev
abirose
imakethesebeatz
bowlthebunny
mmitchelldaviss
paulhirsch
cheeriostweets
tyyylerr
metally
warisfucked
lifeneed
thatsorry
carolehicks
pinsol
bathroomor
ninjamal
thetoughsams
nightlt
northwesternsig
kimota
facibus
jordanmccuaig
wolwol
hatgoing
errorfull
infamousladiit
dearonicajay
kappaomicron
unnoticedbeauty
patterened
kushii
jjdaco
jasonhilimire
daveexplosm
madboarder
erikbussink
whyyy

dutchrudder
wellyes
boylol
bumkittykat
annychih
swearthe
headand
lainas
bgstrowger
claytonkennedy
brittanybosco
lindsaystarbuck
squirrelytonks
wendywings
veneia
liferight
featurerelated
ulovelace
jordannicoler
nadineee
montanaone
sasquatches
couchamp
kaliyahpjones
missrogue
michaelsheen
djurs
sommerland
twinklewhen
prtrice
janetimmins
trinawright
regularolty
venezuelaaaaaaaa
sassyback
geofflloyd
kdubbb
lilxicanita
friendlol
amberlycollins
quotthousand
milesquot
daveracingkites
realityntails
lellos
vegascom
iambranded
peterjdilorenzo
bloggerhappy
cvminc
lobstertony
katost
mrskeith
xstex
amberlily
laterugh
demotix
whoasara
sbmczh
batmansounds
impalaguy
kristymoffatt
ryancoleman
rocketpark
aniinhagriffin
poynterday
dinogoesrawr
todayno
stresswoohoo
taylormcfly
joelfreak
mrvdw
adamselzer
nicei
jonasgirly
articlepodcast
angiasaa
punjoke
pandaran
bryanperson
ronaldtheshort
maloosii
yuhngfresh
djtgutta
trailerdying
mrskidmore
jamesphelps
clintyghost
itiswhatitisent
sazfob
wams
janetconner
xab

# TF2.0 and Keras Model Building

In [26]:
X

array([[   5,  279,    9, ...,    0,    0,    0],
       [  50,   11, 1297, ...,    0,    0,    0],
       [2298, 1085,  655, ...,    0,    0,    0],
       ...,
       [ 934,   22,  923, ...,    0,    0,    0],
       [6788,  125,    7, ...,    0,    0,    0],
       [  89,   12,  208, ...,    0,    0,    0]])

In [27]:
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=42, test_size=0.2, stratify=y)

In [28]:
vec_size= 200

model= Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length, weights=[word_vector_matrix],trainable=False))

model.add(Conv1D(64,8, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))
          
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
          
model.add(Dense(16, activation='relu'))
          
model.add(GlobalMaxPooling1D())
          
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [30]:
%%time
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
100/100 [==============================] - 8s 53ms/step - loss: 0.9572 - accuracy: 0.4928 - val_loss: 0.7112 - val_accuracy: 0.5000
Epoch 2/30
100/100 [==============================] - 5s 50ms/step - loss: 0.7442 - accuracy: 0.5081 - val_loss: 0.7239 - val_accuracy: 0.5038
Epoch 3/30
100/100 [==============================] - 5s 54ms/step - loss: 0.7155 - accuracy: 0.5322 - val_loss: 0.7109 - val_accuracy: 0.5075
Epoch 4/30
100/100 [==============================] - 6s 60ms/step - loss: 0.7085 - accuracy: 0.5250 - val_loss: 0.6969 - val_accuracy: 0.5138s - loss:
Epoch 5/30
100/100 [==============================] - 5s 52ms/step - loss: 0.6886 - accuracy: 0.5559 - val_loss: 0.6916 - val_accuracy: 0.5163
Epoch 6/30
100/100 [==============================] - 5s 53ms/step - loss: 0.6864 - accuracy: 0.5656 - val_loss: 0.6819 - val_accuracy: 0.5312
Epoch 7/30
100/100 [==============================] - 5s 48ms/step - loss: 0.6673 - accuracy: 0.5984 - val_loss: 0.6745 - val_accurac

In [33]:
def get_encode(x):
    x= get_clean_text(x)
    x=token.texts_to_sequences(x)
    x=pad_sequences(x,maxlen=max_length, padding='post')
    return x

In [34]:
get_encode(['hi how are you?'])

array([[318,  78,  27,   7,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0]])

In [41]:
model.predict(get_encode(['thank you for watching']))

array([[0.8292483]], dtype=float32)